In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn import neighbors
from sklearn.preprocessing import MinMaxScaler
from keras.regularizers import l2
from tensorflow.keras import regularizers
from keras.layers import Dropout
import tensorflow as tf
from sklearn.svm import SVR
from sklearn.decomposition import PCA
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
import dateutil
from category_encoders import TargetEncoder
from sklearn import preprocessing
import datetime
from PIL import ImageColor
import math

In [ ]:
training = pd.read_csv("/kaggle/input/ift6758-a20/train.csv")
testing  = pd.read_csv("/kaggle/input/ift6758-a20/test.csv")


training['Location Public Visibility'] = training['Location Public Visibility'].str.lower()
# User Name column is not relevent
training.drop('User Name', 1,inplace=True)
training.drop('User Time Zone', 1,inplace=True)
training.drop('Profile Image', 1,inplace=True)
training.drop('Location', 1,inplace=True)

# mean fill nan for categorical features
training['Profile Cover Image Status'].fillna('Set', inplace=True)
training['UTC Offset'].fillna(training['UTC Offset'].mean(), inplace=True)
training['Profile Theme Color'].fillna(0,inplace = True)
training['Profile Text Color'].fillna(0,inplace = True)
training['Profile Page Color'].fillna(0,inplace = True)

# One hot encodeing some of the categorical variables
training = pd.get_dummies(training, columns = ['Profile Verification Status'])
training = pd.get_dummies(training, columns = ['Location Public Visibility'])
# training = pd.get_dummies(training, columns = ['Profile Category'])

# Binary encoding some Cathegorical variables as binary
training['Personal URL'] = training['Personal URL'].notnull().astype('int')
training['Profile Cover Image Status'] = np.where(training['Profile Cover Image Status']=='Set', 1, 0)
training['Is Profile View Size Customized?'] = training['Is Profile View Size Customized?'].astype(int)
# training['User Language'] = np.where(training['User Language']=='en', 1, 0)

# Mean imputation for 'Avg Daily Profile Clicks'And'Avg Daily Profile Visit Duration in seconds'
imp_median = SimpleImputer( strategy='mean') #for median imputation replace 'mean' with 'median'
imp_median.fit(training[['Avg Daily Profile Clicks','Avg Daily Profile Visit Duration in seconds']])
imputed_train_df = imp_median.transform(training[['Avg Daily Profile Clicks','Avg Daily Profile Visit Duration in seconds']])
imputed_train_df = pd.DataFrame(imputed_train_df,columns=['Avg Daily Profile Clicks','Avg Daily Profile Visit Duration in seconds'])
training[['Avg Daily Profile Clicks','Avg Daily Profile Visit Duration in seconds']] = imputed_train_df[['Avg Daily Profile Clicks','Avg Daily Profile Visit Duration in seconds']]

# target encoding UTC, Profile Category, Language
t_encod_prof_cat=TargetEncoder(cols=['Profile Category']).fit(training['Profile Category'], training['Num of Profile Likes'])
t_encod_prof_utc=TargetEncoder(cols=['UTC Offset']).fit(training['UTC Offset'], training['Num of Profile Likes'])
t_encod_prof_lang=TargetEncoder(cols=['User Language']).fit(training['User Language'], training['Num of Profile Likes'])
training['Profile Category'] = t_encod_prof_cat.transform(training['Profile Category'])
training['UTC Offset'] = t_encod_prof_utc.transform(training['UTC Offset'])
training['User Language'] = t_encod_prof_lang.transform(training['User Language'])



# Log transform of the numerical features
training['Num of People Following'] = np.log(training['Num of People Following'])
training['Num of Status Updates'] = np.log(training['Num of Status Updates'])
training['Avg Daily Profile Clicks'] = np.log(training['Avg Daily Profile Clicks'])
training['Num of Followers'] = np.log(training['Num of Followers'])
training['Avg Daily Profile Visit Duration in seconds'] = np.log(training['Avg Daily Profile Visit Duration in seconds'])
training['Num of Direct Messages'] = np.log(training['Num of Direct Messages'])

# robust scale the nemerical features
training = training.replace([np.inf, -np.inf], 0)
# scaler = preprocessing.RobustScaler()
# robust_scaled_df = scaler.fit_transform(training[['Num of People Following','Num of Status Updates','Avg Daily Profile Clicks','Num of Followers','Avg Daily Profile Visit Duration in seconds','Num of Direct Messages']])
# training[['Num of People Following','Num of Status Updates','Avg Daily Profile Clicks','Num of Followers','Avg Daily Profile Visit Duration in seconds','Num of Direct Messages']] = pd.DataFrame(robust_scaled_df, columns=['Num of People Following','Num of Status Updates','Avg Daily Profile Clicks','Num of Followers','Avg Daily Profile Visit Duration in seconds','Num of Direct Messages'])


# UTF off-set frequency encoding
# enc_time_zone = (training.groupby('UTC Offset').size()) / len(training)
# training['UTC Offset'] = training['UTC Offset'].apply(lambda x : enc_time_zone[x])


# colors encoding, convert to int
#     training['Profile Theme Color'].fillna(0,inplace = True)
#     training['Profile Theme Color']=training['Profile Theme Color'].apply(lambda x: int(str(x), 16))

#     training['Profile Page Color'].fillna(0,inplace = True)
#     training['Profile Page Color']=training['Profile Page Color'].apply(lambda x: int(str(x), 16))

#     training['Profile Text Color'].fillna(0,inplace = True)
#     training['Profile Text Color']=training['Profile Text Color'].apply(lambda x: int(str(x), 16))

# enc_color_theme = (training.groupby('Profile Theme Color').size()) / len(training)
# training['Profile Theme Color'] = training['Profile Theme Color'].apply(lambda x : enc_color_theme[x])

# enc_color_text = (training.groupby('Profile Text Color').size()) / len(training)
# training['Profile Text Color'] = training['Profile Text Color'].apply(lambda x : enc_color_text[x])

# enc_color_page = (training.groupby('Profile Page Color').size()) / len(training)
# training['Profile Page Color'] = training['Profile Page Color'].apply(lambda x : enc_color_page[x])


training["Profile Text Color"].fillna("0084b4",inplace = True)
conv_p_t_c = []
for k in training["Profile Text Color"]:
    if(k != None and k != np.nan and k != 0):
        conv_p_t_c.append(list(ImageColor.getcolor('#'+str(k), "RGB")))
    else:
        conv_p_t_c.append([219, 26, 44])
training[['prof_t_c_r','prof_t_c_b','prof_t_c_g']]=conv_p_t_c

training["Profile Page Color"].fillna("ddeef6",inplace = True)
conv_p_p_c = []
for k in training["Profile Page Color"]:
    if(k != None and k != np.nan and k != 0):
        conv_p_p_c.append(list(ImageColor.getcolor('#'+str(k), "RGB")))
    else:
        conv_p_p_c.append([221, 238, 246])
training[['prof_p_p_r','prof_p_p_b','prof_p_p_g']]=conv_p_p_c

training["Profile Theme Color"].fillna("c0deed",inplace = True)
conv_p_tm_c = []
for k in training["Profile Theme Color"]:
    if(k != None and k != np.nan and k != 0):
        conv_p_tm_c.append(list(ImageColor.getcolor('#'+str(k), "RGB")))
    else:
        conv_p_tm_c.append([192, 222, 237])
training[['prof_p_tm_r','prof_p_tm_b','prof_p_tm_g']]=conv_p_tm_c

training.drop("Profile Text Color", 1,inplace=True)
training.drop("Profile Page Color", 1,inplace=True)
training.drop("Profile Theme Color", 1,inplace=True)




training.set_index('Id',inplace=True)

training['Profile Creation Timestamp'] = training['Profile Creation Timestamp'].apply(lambda x: (dateutil.parser.parse(x).replace(tzinfo=None)-datetime.datetime(1970,1,1)).days)



testing['Location Public Visibility'] = testing['Location Public Visibility'].str.lower()
# User Name column is not relevent
testing.drop('User Name', 1,inplace=True)
testing.drop('User Time Zone', 1,inplace=True)
testing.drop('Profile Image', 1,inplace=True)
testing.drop('Location', 1,inplace=True)

# mean fill nan for categorical features
testing['Profile Cover Image Status'].fillna('Set', inplace=True)
testing['UTC Offset'].fillna(testing['UTC Offset'].mean(), inplace=True)
testing['Profile Theme Color'].fillna(0,inplace = True)
testing['Profile Text Color'].fillna(0,inplace = True)
testing['Profile Page Color'].fillna(0,inplace = True)

# One hot encodeing some of the categorical variables
testing = pd.get_dummies(testing, columns = ['Profile Verification Status'])
testing = pd.get_dummies(testing, columns = ['Location Public Visibility'])
# testing = pd.get_dummies(testing, columns = ['Profile Category'])

# Binary encoding some Cathegorical variables
testing['Personal URL'] = testing['Personal URL'].notnull().astype('int')
testing['Profile Cover Image Status'] = np.where(testing['Profile Cover Image Status']=='Set', 1, 0)
testing['Is Profile View Size Customized?'] = testing['Is Profile View Size Customized?'].astype(int)
# testing['User Language'] = np.where(testing['User Language']=='en', 1, 0)

# Median imputation for 'Avg Daily Profile Clicks'And'Avg Daily Profile Visit Duration in seconds'
imp_median = SimpleImputer( strategy='median') #for median imputation replace 'mean' with 'median'
imp_median.fit(testing[['Avg Daily Profile Clicks','Avg Daily Profile Visit Duration in seconds']])
imputed_train_df = imp_median.transform(testing[['Avg Daily Profile Clicks','Avg Daily Profile Visit Duration in seconds']])
imputed_train_df = pd.DataFrame(imputed_train_df,columns=['Avg Daily Profile Clicks','Avg Daily Profile Visit Duration in seconds'])
testing[['Avg Daily Profile Clicks','Avg Daily Profile Visit Duration in seconds']] = imputed_train_df[['Avg Daily Profile Clicks','Avg Daily Profile Visit Duration in seconds']]

# target encoding utc, location, user language
testing['User Language'] = t_encod_prof_lang.transform(testing['User Language'])
testing['UTC Offset'] = t_encod_prof_utc.transform(testing['UTC Offset'])
testing['Profile Category'] = t_encod_prof_cat.transform(testing['Profile Category'])

# Log transform of the numerical features
testing['Num of People Following'] = np.log(testing['Num of People Following'])
testing['Num of Status Updates'] = np.log(testing['Num of Status Updates'])
testing['Avg Daily Profile Clicks'] = np.log(testing['Avg Daily Profile Clicks'])
testing['Num of Followers'] = np.log(testing['Num of Followers'])
testing['Avg Daily Profile Visit Duration in seconds'] = np.log(testing['Avg Daily Profile Visit Duration in seconds'])
testing['Num of Direct Messages'] = np.log(testing['Num of Direct Messages'])

# robust scale the nemerical features
testing = testing.replace([np.inf, -np.inf], 0)
# scaler = preprocessing.RobustScaler()
# robust_scaled_df = scaler.fit_transform(testing[['Num of People Following','Num of Status Updates','Avg Daily Profile Clicks','Num of Followers','Avg Daily Profile Visit Duration in seconds','Num of Direct Messages']])
# testing[['Num of People Following','Num of Status Updates','Avg Daily Profile Clicks','Num of Followers','Avg Daily Profile Visit Duration in seconds','Num of Direct Messages']] = pd.DataFrame(robust_scaled_df, columns=['Num of People Following','Num of Status Updates','Avg Daily Profile Clicks','Num of Followers','Avg Daily Profile Visit Duration in seconds','Num of Direct Messages'])


# UTF off-set frequency encoding
# enc_time_zone = (testing.groupby('UTC Offset').size()) / len(testing)
# testing['UTC Offset'] = testing['UTC Offset'].apply(lambda x : enc_time_zone[x])


# colors encoding, convert to int
#     testing['Profile Theme Color'].fillna(0,inplace = True)
#     testing['Profile Theme Color']=testing['Profile Theme Color'].apply(lambda x: int(str(x), 16))

#     testing['Profile Page Color'].fillna(0,inplace = True)
#     testing['Profile Page Color']=testing['Profile Page Color'].apply(lambda x: int(str(x), 16))

#     testing['Profile Text Color'].fillna(0,inplace = True)
#     testing['Profile Text Color']=testing['Profile Text Color'].apply(lambda x: int(str(x), 16))

# enc_color_theme = (testing.groupby('Profile Theme Color').size()) / len(testing)
# testing['Profile Theme Color'] = testing['Profile Theme Color'].apply(lambda x : enc_color_theme[x])

# enc_color_text = (testing.groupby('Profile Text Color').size()) / len(testing)
# testing['Profile Text Color'] = testing['Profile Text Color'].apply(lambda x : enc_color_text[x])

# enc_color_page = (testing.groupby('Profile Page Color').size()) / len(testing)
# testing['Profile Page Color'] = testing['Profile Page Color'].apply(lambda x : enc_color_page[x])

testing["Profile Text Color"].fillna("0084b4",inplace = True)
conv_p_t_ct = []
for k in testing["Profile Text Color"]:
    if(k != None and k != np.nan and k != 0):
        conv_p_t_ct.append(list(ImageColor.getcolor('#'+str(k), "RGB")))
    else:
        conv_p_t_ct.append([219, 26, 44])
testing[['prof_t_c_r','prof_t_c_b','prof_t_c_g']]=conv_p_t_ct

testing["Profile Page Color"].fillna("ddeef6",inplace = True)
conv_p_p_ct = []
for k in testing["Profile Page Color"]:
    if(k != None and k != np.nan and k != 0):
        conv_p_p_ct.append(list(ImageColor.getcolor('#'+str(k), "RGB")))
    else:
        conv_p_p_ct.append([221, 238, 246])
testing[['prof_p_p_r','prof_p_p_b','prof_p_p_g']]=conv_p_p_ct

testing["Profile Theme Color"].fillna("c0deed",inplace = True)
conv_p_tm_ct = []
for k in testing["Profile Theme Color"]:
    if(k != None and k != np.nan and k != 0):
        conv_p_tm_ct.append(list(ImageColor.getcolor('#'+str(k), "RGB")))
    else:
        conv_p_tm_ct.append([192, 222, 237])
testing[['prof_p_tm_r','prof_p_tm_b','prof_p_tm_g']]=conv_p_tm_ct

testing.drop("Profile Text Color", 1,inplace=True)
testing.drop("Profile Page Color", 1,inplace=True)
testing.drop("Profile Theme Color", 1,inplace=True)


testing.set_index('Id',inplace=True)

testing['Profile Creation Timestamp'] = testing['Profile Creation Timestamp'].apply(lambda x: (dateutil.parser.parse(x).replace(tzinfo=None)-datetime.datetime(1970,1,1)).days)


training.astype('object').describe().transpose()



In [ ]:
y_col = ['Num of Profile Likes']
x_col = training.columns.difference(['Num of Profile Likes'])
x_col

In [ ]:
# pearson's correlation feature selection 
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
# define feature selection
fs = SelectKBest(score_func=f_regression, k='all')
# apply feature selection
X_selected = fs.fit_transform(training[x_col], training[y_col].values.ravel())
names = training[x_col].columns.values[fs.get_support()]
scores = fs.scores_[fs.get_support()]
names_scores = list(zip(names, scores))
ns_df = pd.DataFrame(data = names_scores, columns=['Feat_names', 'F_Scores'])
#Sort the dataframe for better visualization
ns_df_sorted = ns_df.sort_values(['F_Scores', 'Feat_names'], ascending = [False, True])
print(ns_df_sorted)

In [ ]:
training.astype('object').describe().transpose()

In [ ]:
training.drop(labels =['Profile Cover Image Status','Location Public Visibility_??'], axis =1,inplace=True)
testing.drop(labels =['Profile Cover Image Status','Location Public Visibility_??'], axis =1,inplace=True)
# training.drop('Profile Category_', 1,inplace=True)
# training.drop('Profile Cover Image Status', 1,inplace=True)

# testing.drop('Location Public Visibility_??', 1,inplace=True)
# # testing.drop('Profile Category_', 1,inplace=True)
# testing.drop('Profile Cover Image Status', 1,inplace=True)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( np.array(training[x_col]), np.array(training[y_col]), test_size=0.20, random_state=42)

In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error as MSE 
from catboost import CatBoostRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import RidgeCV
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor

# https://www.datacamp.com/community/tutorials/xgboost-in-python
xg_reg = make_pipeline(RobustScaler(),xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.01,
                max_depth = 7, alpha = 10, n_estimators = 500,gamma=0.005, min_child_weight=1))

cat_reg = make_pipeline(RobustScaler(),CatBoostRegressor(verbose=0, n_estimators=890,learning_rate=0.01,depth=7))
grad_reg = GradientBoostingRegressor(n_estimators=500,learning_rate=0.01)
# kernel_reg = make_pipeline(StandardScaler(), KernelRidge())
regr = make_pipeline(RobustScaler(), SVR(C=9, epsilon=0.2))


# estimators = [
#      ('lr', RidgeCV()),
#      ('svr', LinearSVR(random_state=42)),
#      ('svrnl', SVR(C=9, epsilon=0.2))
# ]
# stack_reg = make_pipeline(RobustScaler(), StackingRegressor(
#     estimators=estimators,
#     final_estimator=RandomForestRegressor(n_estimators=10, random_state=42)))
# stack_reg.fit(X_train, y_train)



# Random forest regressor  best: 2.364, e=2, d =10
# best_e=0
# best_d=0
# best_score=1000
# for e in [2,4,8,16,32,64,128,256,512,1024]:
#     for d in [2,3,4,5,6,7,8,9,10]: 
#         print(e,d)
#         rand_reg =  RandomForestRegressor(max_depth=d, n_estimators=e, random_state=0)
#         rand_reg.fit(X_train, y_train)
#         y_pred = rand_reg.predict(X_test)
#         for i in range(len(y_test)):
#         #     y_pred[i]=y_pred[i]
#             if(y_pred[i]<0):
#                 y_pred[i]=5390
#         rmse = np.sqrt(mean_squared_log_error(y_test, y_pred))
#         print("RMSE : % f" %(rmse))
#         if(rmse<best_score):
#             best_score = rmse
#             best_e = e
#             best_d = d
#         print("RMSE : % f" %(best_score))
#         print(best_e)
#         print(best_d)
#         print()
         


# for c in [1,2,3,4,5,6,7,8,9]:
#     for s in [2,5,10,20,40,60,100]:
#         print(c,s)
#         lsvr = LinearSVR(random_state=s, C=c)
#         lsvr.fit(X_train, y_train)

#         y_pred = lsvr.predict(X_test)
#         for i in range(len(y_test)):
#             y_pred[i]=y_pred[i]
#             if(y_pred[i]<0):
#                 y_pred[i]=5390
    

# rmse = np.sqrt(mean_squared_log_error(y_test, y_pred)) 
# print("RMSE : % f" %(rmse)) 

# for c in ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed']:
#     print(c)
#     regr = make_pipeline(RobustScaler(), SVR(kernel=c,C=9, epsilon=0.1))
#     regr.fit(X_train, y_train)
#     y_pred = regr.predict(X_test)
#     for i in range(len(y_test)):
#         if(y_pred[i]<0):
#             y_pred[i]=5390
#         y_pred[i]=y_pred[i]

#     rmse = np.sqrt(mean_squared_log_error(y_test, y_pred)) 
#     print("RMSE : % f" %(rmse)) 



# minscore = 500
# vec = np.zeros(2)
# for m in range(500):
#     randvec = np.random.rand(2)
#     randvec = randvec / np.sum(randvec)
#     er = VotingRegressor([('xg', xg_reg),('svr',regr)],weights=randvec)
#     er.fit(X_train, y_train)

#     y_pred = er.predict(X_test)

#     for i in range(len(y_test)):
#         if(y_pred[i]<0):
#             y_pred[i]=5390
#         y_pred[i]=y_pred[i]

#     rmse = np.sqrt(mean_squared_log_error(y_test, y_pred)) 
#     print("RMSE : % f" %(rmse)) 
#     print(randvec)
#     if(minscore > rmse):
#         minscore = rmse
#         vec = randvec
#     print(minscore)
#     print(vec)


# training model:

rand_reg =  RandomForestRegressor(max_depth=10, n_estimators=2, random_state=0)

rand_reg.fit(X_train, y_train)
y_pred = rand_reg.predict(X_test)
for i in range(len(y_test)):
#     y_pred[i]=y_pred[i]
    if(y_pred[i]<0):
        y_pred[i]=5390
rmse = np.sqrt(mean_squared_log_error(y_test, y_pred)) 
print("RMSE : % f" %(rmse)) 



[0.44153715 0.67033311 0.7148971 ]
2.6197931110765706

In [ ]:
count=0
for i in range(len(y_test)):
    if(y_test[i]<0):
        y_test[i]=0


In [ ]:
y_pred

In [ ]:
y_test

In [ ]:
# params = {"objective":"reg:squarederror",'colsample_bytree': 0.3,'learning_rate': 0.01,
#                 'max_depth': 7, 'alpha': 10}

# cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,
#                     num_boost_round=50,early_stopping_rounds=10,metrics="rmse", as_pandas=True, seed=123)
kfold = KFold(n_splits=10, random_state=7)
results = cross_val_score(er, X_train, y_train, cv=kfold)
print(results)

In [ ]:
testing.astype('object').describe().transpose()

In [ ]:
y_pred = rand_reg.predict(np.array(testing[x_col]))
for i in range(len(y_pred)):
    if(y_pred[i]<0):
        y_pred[i]=5390
dummy = pd.DataFrame()
dummy["Id"] = testing.index
dummy["Predicted"]=y_pred
dummy.to_csv("submission.csv", sep=',' ,index=False)
